In [1]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['./cnn_finetune'])

from load_cifar import load_cifar10_data, load_cifar100_data

import numpy as np; print('numpy:', np.__version__)
import sklearn; print('sklearn:', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import keras; print('keras', keras.__version__)

Python 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609] on linux


Using TensorFlow backend.


numpy: 1.16.2
sklearn: 0.20.3
tensorflow 1.13.1
keras 2.2.4


In [2]:
import tensorflow as tf; print('tensorflow', tf.__version__)
import keras; print('keras', keras.__version__)

tensorflow 1.13.1
keras 2.2.4


In [3]:
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

config = tf.ConfigProto()

# cpu
# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores, inter_op_parallelism_threads=num_cores, allow_soft_placement=True, device_count={'CPU': 4})
    
# gpu allow_growth
config.gpu_options.allow_growth = True

# set memory limit of gpu
# config.gpu_options.per_process_gpu_memory_fraction = 0.5

set_session(tf.Session(config=config))

In [4]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger

from cnn_finetune.resnet_152 import resnet152_model
from cnn_finetune.load_cifar import load_cifar10_data, load_cifar100_data

In [5]:
# Example to fine-tune on 3000 samples from Cifar10
model_name = 'resnet_152'
dataset_name = 'imaterialist'
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 2019
batch_size = 24
epochs = 100
load_weights = True
fine_tuning = True

# Load our model
model = resnet152_model(img_rows, img_cols, channel, num_classes, fine_tuning, load_weights)

Instructions for updating:
Colocations handled automatically by placer.
resnet152_model: classes: 2019 fine-tuning: True
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2 (None, 230, 230, 3)  0           data[0][0]                       
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9408        conv1_zeropadding[0][0]          
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]           

__________________________________________________________________________________________________
res5b_branch2b_zeropadding (Zer (None, 9, 9, 512)    0           res5b_branch2a_relu[0][0]        
__________________________________________________________________________________________________
res5b_branch2b (Conv2D)         (None, 7, 7, 512)    2359296     res5b_branch2b_zeropadding[0][0] 
__________________________________________________________________________________________________
bn5b_branch2b (BatchNormalizati (None, 7, 7, 512)    2048        res5b_branch2b[0][0]             
__________________________________________________________________________________________________
scale5b_branch2b (Scale)        (None, 7, 7, 512)    1024        bn5b_branch2b[0][0]              
__________________________________________________________________________________________________
res5b_branch2b_relu (Activation (None, 7, 7, 512)    0           scale5b_branch2b[0][0]           
__________

In [6]:
from lib.DataGenerator import DataGenerator
val_data_generator = DataGenerator(file_type='val', corp=True, batch_size=batch_size)
train_data_generator = DataGenerator(file_type='train', corp=True, batch_size=batch_size)

Found 10095 images belonging to 2019 classes.
Found 1008641 images belonging to 2019 classes.


In [ ]:
steps_per_epoch = len(train_data_generator)
validation_steps = len(val_data_generator)

In [ ]:
print(steps_per_epoch)
print(validation_steps)

63041
631


In [ ]:
# Learning rate is changed to 0.001
# adam = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
adam = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, min_delta=1e-8, decay=0.0, amsgrad=False)
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])

checkpoint = ModelCheckpoint("./models/" + model_name + "/"+ dataset_name +"-e" + "{epoch:02d}-{val_loss:.4f}.hd5", 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=True, 
                             mode='auto', 
                             period=1)

checkpoint_best = ModelCheckpoint("./models/" + model_name + "/"+ dataset_name + "-best.hd5", 
                                  monitor='val_loss', 
                                  verbose=1, 
                                  save_best_only=True, 
                                  save_weights_only=True,
                                  mode='auto', 
                                  period=1)

earlyStopping = EarlyStopping(monitor='val_loss', 
                              min_delta=0, 
                              patience=10, 
                              verbose=1, 
                              mode='auto', 
                              baseline=None, 
                              restore_best_weights=False)

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

tensorBoard = TensorBoard(log_dir='./models/' + model_name +'/logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

csvLogger = CSVLogger(filename='./models/'+ model_name + '/' + dataset_name + "-log.csv", separator=',', append=False)


/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [ ]:
model.fit_generator(train_data_generator,
          steps_per_epoch = steps_per_epoch,
          epochs=epochs,
          shuffle=True,
          verbose=1,
          validation_data=val_data_generator,
          validation_steps=validation_steps,
          workers=100,
          use_multiprocessing=True,
          callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger]
          )

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
63041/63041 [==============================] - 26830s 426ms/step - loss: 6.8413 - acc: 0.0294 - val_loss: 5.4681 - val_acc: 0.0774

Epoch 00001: saving model to ./models/resnet_152/imaterialist-e01-5.4681.hd5

Epoch 00001: val_loss improved from inf to 5.46813, saving model to ./models/resnet_152/imaterialist-best.hd5
Epoch 2/100
63041/63041 [==============================] - 26277s 417ms/step - loss: 4.7484 - acc: 0.2189 - val_loss: 3.9481 - val_acc: 0.2386

Epoch 00002: saving model to ./models/resnet_152/imaterialist-e02-3.9481.hd5

Epoch 00002: val_loss improved from 5.46813 to 3.94813, saving model to ./models/resnet_152/imaterialist-best.hd5
Epoch 3/100
 3533/63041 [>.............................] - ETA: 6:52:12 - loss: 3.3576 - acc: 0.3893

In [ ]:
# Make predictions
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)
print(Y_valid.shape, predictions_valid.shape)

In [ ]:
# import numpy as np
# np.set_printoptions(threshold=np.inf)
# print(Y_valid)
# print(predictions_valid)

In [ ]:
print('cross-entropy:', sklearn.metrics.log_loss(Y_valid, predictions_valid))

y_true = np.argmax(Y_valid, axis=1)
y_hat = np.argmax(predictions_valid, axis=1)
# print('report:', sklearn.metrics.classification_report(y_true, y_hat))
print('acc:', sklearn.metrics.accuracy_score(y_true, y_hat))
# print('roc:', sklearn.metrics.roc_auc_score(y_true, y_hat))
# print('confusion_matrix:\n', sklearn.metrics.confusion_matrix(y_true, y_hat))